**Tax Assistant Lite is an AI‑powered tool for small CPA firms that automates transaction categorization, flags potential deductions, and generates client‑friendly tax summaries. Clients securely upload CSV files, and the system organizes them in a structured Google Drive folder by client, year, and month, producing both on‑screen insights and downloadable PDF reports.**

/taxassistantlite
    /client
        /client_pseudo
            /2025
                /february
                    raw_uploads/
                        2025-02-15_transactions.csv
                    processed_reports/
                        2025-02-15_tax_report.pdf


In [3]:
# === 1. Install dependencies ===
!pip install pandas gradio fpdf transformers datasets accelerate --quiet

  Preparing metadata (setup.py) ... done


In [2]:
from google.colab import drive
import os
from datetime import datetime

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define base path with 'client'
base_path = '/content/drive/MyDrive/taxassistantlite/clients/client_pseudo/2025/february'

# 3. Create subfolders if they don't exist
raw_path = os.path.join(base_path, 'raw_uploads')
processed_path = os.path.join(base_path, 'processed_reports')

os.makedirs(raw_path, exist_ok=True)
os.makedirs(processed_path, exist_ok=True)

print(f"Raw uploads folder: {raw_path}")
print(f"Processed reports folder: {processed_path}")

# 4. Example: Save an uploaded CSV to the correct folder
def save_uploaded_csv(uploaded_file):
    today = datetime.today().strftime('%Y-%m-%d')
    save_path = os.path.join(raw_path, f"{today}_transactions.csv")
    with open(save_path, 'wb') as f:
        f.write(uploaded_file.read())
    print(f"File saved to: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Raw uploads folder: /content/drive/MyDrive/taxassistantlite/client/client_pseudo/2025/february/raw_uploads
Processed reports folder: /content/drive/MyDrive/taxassistantlite/client/client_pseudo/2025/february/processed_reports


In [10]:
import pandas as pd
import gradio as gr
from fpdf import FPDF
import re

# --- Keyword banks ---
business_keywords = [
    "purchase", "purchase of", "bought", "buy", "acquired", "supplies", "materials",
    "consumables", "equipment", "tools", "parts", "components", "accessories",
    "maintenance", "repair", "service", "subscription", "license", "software",
    "training", "course", "seminar", "conference", "certification",
    "travel", "mileage", "parking", "toll", "hotel", "accommodation", "transport", "delivery",
    "insurance", "premium", "rent", "lease", "utilities", "internet", "phone",
    "postage", "courier", "shipping", "packaging", "printing", "marketing", "advertising",
    "inventory", "stock", "merchandise", "goods", "raw", "ingredient", "safety", "ppe"
]

personal_keywords = [
    "mortgage", "mortgage interest", "property tax", "real estate tax",
    "tuition", "student loan interest", "education expense", "medical expense",
    "donation", "charity", "uniform", "tools", "supplies", "home office",
    "internet", "phone", "continuing education", "childcare", "energy credit"
]

# --- Non-deductible patterns ---
NON_DEDUCTIBLE_PATTERNS = [
    r"\bowner(?:'s)?\s*draw\b", r"\bdividend\b", r"\bloan\s*repay(?:ment)?\b",
    r"\bpersonal\b", r"\bfine\b", r"\bpenalty\b", r"\btraffic\b", r"\bspeeding\b",
    r"\bincome\s*tax\b", r"\bcorporation\s*tax\b", r"\birpf\b", r"\biva\b", r"\bwithholding\b"
]

def _is_non_deductible(text: str) -> bool:
    return any(re.search(pat, text) for pat in NON_DEDUCTIBLE_PATTERNS)

# --- Categorization ---
def categorize_transactions(df):
    categories = []
    for desc in df['Description'].str.lower():
        if any(w in desc for w in ["travel", "hotel", "taxi", "uber", "bus", "train", "flight"]):
            categories.append("Travel")
        elif any(w in desc for w in ["purchase", "buy", "acquired", "supplies", "materials", "tools", "equipment"]):
            categories.append("Supplies/Materials")
        elif any(w in desc for w in ["software", "subscription", "license"]):
            categories.append("Software")
        elif any(w in desc for w in ["training", "course", "seminar", "conference", "certification"]):
            categories.append("Training")
        elif any(w in desc for w in ["client payment", "invoice", "sale"]):
            categories.append("Income")
        else:
            categories.append("Other")
    df['Category'] = categories
    return df

# --- Deduction detection ---
def find_deductions(df, profile_type="business"):
    if profile_type == "business":
        keywords = set(business_keywords)
    elif profile_type == "personal":
        keywords = set(personal_keywords)
    else:
        keywords = set()

    flags = []
    for _, row in df.iterrows():
        amt = float(row.get('Amount', 0) or 0)
        text = f"{str(row.get('Category', ''))} {str(row.get('Description', ''))}".lower()

        # Treat any non-income as expense
        if amt > 0 and row['Category'].lower() != "income":
            amt = -amt

        if amt < 0 and not _is_non_deductible(text) and any(kw in text for kw in keywords):
            flags.append("Potential Deduction")
        else:
            flags.append("")
    df["Deduction_Flag"] = flags
    return df

# --- Summary ---
def generate_summary(df):
    flagged = df['Deduction_Flag'].astype(str).str.strip() != ""
    total_expenses = df[flagged]['Amount'].sum()
    return f"Total potential deductible expenses: {total_expenses:.2f} USD"

# --- PDF ---
def create_pdf(summary, df):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, "Tax Assistant Lite Report\n")
    pdf.multi_cell(0, 10, summary)
    pdf.multi_cell(0, 10, "\nTransactions:\n")
    for _, row in df.iterrows():
        pdf.multi_cell(0, 8, f"{row['Date']} - {row['Description']} - {row['Amount']} USD - {row['Category']} - {row['Deduction_Flag']}")
    pdf.output("tax_report.pdf")
    return "tax_report.pdf"

# --- Main process ---
def process_file(file, profile_type):
    df = pd.read_csv(file.name)
    df = categorize_transactions(df)
    df = find_deductions(df, profile_type=profile_type)
    summary = generate_summary(df)
    pdf_path = create_pdf(summary, df)
    return summary, pdf_path

# --- Gradio UI ---
demo = gr.Interface(
    fn=process_file,
    inputs=[
        gr.File(label="Upload CSV with Date, Description, Amount"),
        gr.Radio(["business", "personal"], label="Profile Type")
    ],
    outputs=[
        gr.Textbox(label="Summary"),
        gr.File(label="Download PDF Report")
    ],
    title="Tax Assistant Lite",
    description="Upload your transactions CSV to get AI categorization, deduction flags, and a PDF report."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://688411fbbf7265d322.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
